In [ ]:
EHStreamJsonDF.selectExpr(
                  "tripId"\
                  ,"createdAt"\
                  ,"startLocation"\
                  ,"endLocation"\
                  ,"distance"\
                  ,"fare")\
.writeStream.format("delta")\
.outputMode("append")\
.option("checkpointLocation", "dbfs:/tripsCheckpointLocation1/")\
.option("mergeSchema", "true")\
.start("dbfs:/TripsEventHubDelta1")

In [ ]:
EHStreamJsonDF.groupBy(window('createdAt',"1 minutes"),'startLocation').count().orderBy('window') \
.writeStream.format("delta") \
.outputMode("complete") \
.option("truncate", "false") \
.option("checkpointLocation", "dbfs:/tripsCheckpointLocationTumbling2/") \
.option("mergeSchema", "true") \
.start("dbfs:/TripsEventHubDeltaTumbling2") 

In [ ]:
EHStreamJsonDF.groupBy(window('createdAt',"1 minutes"),'startLocation').count().orderBy('window')\
.writeStream.format("memory") \
.outputMode("complete") \
.option("truncate", "false") \
.option("checkpointLocation", "dbfs:/tripsCheckpointLocation/") \
.queryName("TripsTumblingQuery").start() 

In [ ]:
%sql
SELECT * FROM TripsTumblingQuery ORDER BY Window desc

In [ ]:
%sql
DROP TABLE IF EXISTS TripsAggTumbling;
CREATE TABLE IF NOT EXISTS TripsAggTumbling
USING DELTA
LOCATION "dbfs:/TripsEventHubDeltaTumbling2/"

In [ ]:
%sql
SELECT * FROM TripsAggTumbling ORDER BY Window desc